In [1]:
from pystac_client import Client
from utils import (
    read_url,
    UserSettings,
    coords_to_image,
    process_row,
    get_asset_by_common_name,
    convert_coordinates,
)
from pystac import Asset, read_file
from urllib.parse import urlparse
import geopandas as gpd
import pandas as pd
import boto3, botocore
import io
import os
from pystac.stac_io import DefaultStacIO, StacIO
import rasterio
from shapely.wkt import loads
from pystac import Item
import pyproj
import numpy as np
from loguru import logger

# os.environ["GDAL_DATA"] = "/opt/conda/envs/env_label/share/gdal"
# os.environ["PROJ_LIB"] = "/opt/conda/envs/env_label/share/proj"

In [2]:
stac_endpoint = "https://stac-api-dev.terradue.com/"

headers = []

cat = Client.open(stac_endpoint, headers=headers, ignore_conformance=True)
cat

/opt/conda/envs/env_label/lib/python3.10/site-packages/pystac_client/client.py:177: FutureWarning: The `ignore_conformance` option is deprecated and will be removed in the next major release. Instead use `set_conforms_to` or `add_conforms_to` to control behavior.
  warnings.warn(


<Client id=stac-fastapi>

In [3]:
collections = ["ai-extensions-svv-dataset-labels"]

query = cat.search(collections=collections)

In [4]:
[item.get_assets()["labels"] for item in query.item_collection()]

[<Asset href=s3://argo-wfs/svv-dataset/S2B_10SFH_20230613_0_L2A/label-S2B_10SFH_20230613_0_L2A.geojson>,
 <Asset href=s3://argo-wfs/svv-dataset/S2B_10SFG_20230613_0_L2A/label-S2B_10SFG_20230613_0_L2A.geojson>,
 <Asset href=s3://argo-wfs/svv-dataset/S2A_10SFH_20230618_0_L2A/label-S2A_10SFH_20230618_0_L2A.geojson>,
 <Asset href=s3://argo-wfs/svv-dataset/S2A_11SKB_20230618_0_L2A/label-S2A_11SKB_20230618_0_L2A.geojson>,
 <Asset href=s3://argo-wfs/svv-dataset/S2A_10SGG_20230618_0_L2A/label-S2A_10SGG_20230618_0_L2A.geojson>,
 <Asset href=s3://argo-wfs/svv-dataset/S2A_10SFG_20230618_0_L2A/label-S2A_10SFG_20230618_0_L2A.geojson>]

In [5]:
settings = UserSettings("usersettings.json")

settings.set_s3_environment(
    query.item_collection()[0].get_assets()["labels"].get_absolute_href()
)

print(os.environ["AWS_ACCESS_KEY_ID"])

SCWKDBM3QX8BXWW3RB6E


In [6]:
StacIO.set_default(DefaultStacIO)

In [7]:
label_item = query.item_collection()[0]
label_item

<Item id=label_S2B_10SFH_20230613_0_L2A>

In [8]:
label_item.get_assets()

{'labels': <Asset href=s3://argo-wfs/svv-dataset/S2B_10SFH_20230613_0_L2A/label-S2B_10SFH_20230613_0_L2A.geojson>}

In [9]:
# create normalized difference function
def nd(a, b):
    return (a - b) / (a + b)


def read_geojson(
    label_item: Item, user_settings="usersettings.json", asset_key="labels"
):
    settings = UserSettings(user_settings)

    settings.set_s3_environment(label_item.get_assets()[asset_key].get_absolute_href())
    session = botocore.session.Session()

    s3_client = session.create_client(
        service_name="s3",
        region_name=os.environ.get("AWS_REGION"),
        use_ssl=True,
        endpoint_url=os.environ.get("AWS_S3_ENDPOINT"),
        aws_access_key_id=os.environ.get("AWS_ACCESS_KEY_ID"),
        aws_secret_access_key=os.environ.get("AWS_SECRET_ACCESS_KEY"),
    )

    parsed = urlparse(label_item.get_assets()[asset_key].get_absolute_href())

    bucket = parsed.netloc
    key = parsed.path[1:]

    obj = s3_client.get_object(Bucket=bucket, Key=key)

    return gpd.read_file(io.BytesIO(obj["Body"].read()))


def sample_data(label_item, source_item=None, common_bands=["red", "nir"]):
    gdf = read_geojson(label_item)

    if source_item is None:
        source_item = read_file(
            [link.target for link in label_item.get_links() if link.rel in ["source"]][
                0
            ]
        )

    dataset = {}
    for common_band in common_bands:
        logger.info(f"Reading {common_band} band")
        dataset[common_band] = rasterio.open(
            get_asset_by_common_name(source_item, common_band).get_absolute_href()
        )

    def convert_row(row, target_crs):
        "EPSG:4326"
        longitude = row.geometry.x
        latitude = row.geometry.y

        src_crs = "EPSG:4326"

        row["utm_x"], row["utm_y"] = convert_coordinates(
            src_crs, target_crs, longitude, latitude
        )

        return pd.Series(row)

    crs_info = dataset[common_bands[0]].crs
    target_crs = f"EPSG:{crs_info.to_epsg()}"
    gdf = gdf.apply(convert_row, target_crs=target_crs, axis=1)

    points_utm = [(x, y) for x, y in zip(gdf["utm_x"], gdf["utm_y"])]

    for common_band in common_bands:
        logger.info(f"Sampling {common_band} band")
        gdf[common_band] = [
            val[0] / 10000 for val in dataset[common_band].sample(points_utm, 1)
        ]

    if "red" in common_bands and "nir" in common_bands:
        gdf["ndvi"] = nd(gdf["nir"], gdf["red"])
    if "green" in common_bands and "nir" in common_bands:
        gdf["ndwi1"] = nd(gdf["green"], gdf["nir"])
    if "nir" in common_bands and "swir16" in common_bands:
        gdf["ndwi2"] = nd(gdf["nir"], gdf["swir16"])

    return gdf

In [10]:
tmp_gdfs = []

for label_item in query.item_collection():

    sampled_data = sample_data(label_item=label_item, common_bands=["coastal", "red", "green", "blue", "nir", "nir08", "nir09", "swir16", "swir22"])
    
    tmp_gdfs.append(sampled_data)

gdf = pd.concat(tmp_gdfs)


2023-08-04 05:55:51.347 | INFO     | __main__:sample_data:45 - Reading coastal band
2023-08-04 05:55:53.840 | INFO     | __main__:sample_data:45 - Reading red band
2023-08-04 05:55:55.560 | INFO     | __main__:sample_data:45 - Reading green band
2023-08-04 05:55:57.270 | INFO     | __main__:sample_data:45 - Reading blue band
2023-08-04 05:55:59.067 | INFO     | __main__:sample_data:45 - Reading nir band
2023-08-04 05:56:00.814 | INFO     | __main__:sample_data:45 - Reading nir08 band
2023-08-04 05:56:02.525 | INFO     | __main__:sample_data:45 - Reading nir09 band
2023-08-04 05:56:04.242 | INFO     | __main__:sample_data:45 - Reading swir16 band
2023-08-04 05:56:06.022 | INFO     | __main__:sample_data:45 - Reading swir22 band
2023-08-04 05:56:09.576 | INFO     | __main__:sample_data:70 - Sampling coastal band
2023-08-04 05:56:22.889 | INFO     | __main__:sample_data:70 - Sampling red band
2023-08-04 05:57:42.121 | INFO     | __main__:sample_data:70 - Sampling green band
2023-08-04 05:

In [13]:
gdf.describe()

,utm_x,utm_y,coastal,red,green,blue,nir,nir08,nir09,swir16,swir22,ndvi,ndwi1,ndwi2
count,1534.000000,1.534000e+03,1534.000000,1534.000000,1534.000000,1534.000000,1534.000000,1534.000000,1534.000000,1534.000000,1534.000000,1534.000000,1534.000000,1534.000000
mean,606369.687313,4.187560e+06,0.064559,0.109282,0.099238,0.077235,0.194373,0.206784,0.208543,0.193951,0.139377,0.197840,-0.219207,0.010196
std,154536.548895,5.366458e+04,0.050475,0.066830,0.055794,0.055445,0.114833,0.121875,0.117879,0.119755,0.086418,0.335595,0.369345,0.186601
min,200009.978139,4.090250e+06,0.000200,0.003600,0.008800,0.000100,0.000100,0.000100,0.000100,0.003000,0.003100,-0.973333,-0.845459,-0.987097
25%,615859.964959,4.148850e+06,0.036200,0.062425,0.066400,0.045050,0.080400,0.082325,0.090025,0.079525,0.064825,-0.068093,-0.505200,-0.109481
50%,655189.998843,4.186600e+06,0.052700,0.096550,0.089200,0.067000,0.216500,0.234800,0.240450,0.192950,0.129450,0.197909,-0.349640,-0.006430
75%,689600.010583,4.223425e+06,0.074450,0.146000,0.114550,0.091375,0.273400,0.294075,0.290400,0.296575,0.204650,0.431809,0.108895,0.083600
max,753550.029670,4.299910e+06,0.708000,0.742400,0.732800,0.707600,0.729600,0.735400,0.687500,0.617900,0.491000,0.926432,0.991111,0.859485


In [12]:
gdf.to_pickle('sprint-0-STAC-labels-to-dataframe.pkl')